In [6]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense,Dropout
import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import numpy as np


In [9]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("abdelghaniaaba/wildfire-prediction-dataset")

print("Path to dataset files:", path)

Path to dataset files: /root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1


In [10]:
train_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/train"
valid_dir = r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/valid"
test_dir =r"/root/.cache/kagglehub/datasets/abdelghaniaaba/wildfire-prediction-dataset/versions/1/test"

#set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

#load images from directories
train_generator = train_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = valid_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = test_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

Found 30250 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.
Found 6300 images belonging to 2 classes.


In [13]:
#building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid') #binary classification: wildfire or no wildfire
])

#compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [15]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [17]:
history=model.fit(train_generator,validation_data=valid_generator ,epochs=5,verbose =1)

Epoch 1/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 239s 252ms/step - accuracy: 0.9376 - loss: 0.1706 - val_accuracy: 0.9489 - val_loss: 0.1414
Epoch 2/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 240s 254ms/step - accuracy: 0.9455 - loss: 0.1507 - val_accuracy: 0.9502 - val_loss: 0.1295
Epoch 3/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 258s 249ms/step - accuracy: 0.9515 - loss: 0.1349 - val_accuracy: 0.9548 - val_loss: 0.1198
Epoch 4/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 233s 246ms/step - accuracy: 0.9592 - loss: 0.1107 - val_accuracy: 0.9568 - val_loss: 0.1154
Epoch 5/5
946/946 ━━━━━━━━━━━━━━━━━━━━ 231s 244ms/step - accuracy: 0.9623 - loss: 0.1028 - val_accuracy: 0.9617 - val_loss: 0.1066


In [18]:
model.save('ffd_model.h5')

In [2]:

import tkinter as tk
from tkinter import filedialog
from PIL import Image, ImageTk
import tensorflow as tf
from tensorflow.keras.models import load_model




In [25]:
model=load_model('ffd_model.h5')
print("Model loaded successfully")

Model loaded successfully


In [3]:
!pip install gradio==3.35.2

INFO: pip is looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of mdit-py-plugins to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.7/19.7 MB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.5/84.5 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.5/50.5 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.3/62.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 

In [7]:
import gradio as gr
import tensorflow as tf
from tensorflow.keras.models import load_model
from PIL import Image
import numpy as np

model = load_model('ffd_model.h5')

def predict_image(img):
    img = img.resize((64, 64))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    prediction = model.predict(img_array)[0][0]
    result = "Wildfire" if prediction > 0.5 else "No Wildfire"
    return result

iface = gr.Interface(
    fn=predict_image,
    inputs=gr.Image(type="pil"),  # Updated input definition
    outputs="text",
    title="Forest Fire Detection",
    description="Upload an image to predict whether it contains a wildfire."
)

iface.launch()

IMPORTANT: You are using gradio version 3.35.2, however version 4.44.1 is available, please upgrade.
--------
Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Note: opening Chrome Inspector may crash demo inside Colab notebooks.

To create a public link, set `share=True` in `launch()`.


<IPython.core.display.Javascript object>